In [1]:
import requests
import json
import numpy as np
import pandas as pd
import csv
import time
from datetime import datetime

## visualizacion.csv

In [3]:
visualizacion_df = pd.read_csv('agrupamiento_linea.csv', sep=";", header=0)
bicis_df = pd.read_csv('bicimad_stations.csv', sep=";", header=0)
lineas_df = pd.read_csv('line_headers_info.csv', sep=";", header=0)

In [4]:
bicis_df.rename(columns={'id': 'idunplug_station','name': 'name_origen'}, inplace=True)
visualizacion_df =  visualizacion_df.merge(bicis_df[['idunplug_station','name_origen','longitude','latitude']], on='idunplug_station',how='left')
visualizacion_df.rename(columns={'longitude': 'longitud_origen','latitude': 'latitud_origen'}, inplace=True)
bicis_df.rename(columns={'idunplug_station': 'idplug_station','name_origen': 'name_destino'}, inplace=True)
visualizacion_df =  visualizacion_df.merge(bicis_df[['idplug_station','name_destino','longitude','latitude']], on='idplug_station',how='left')
visualizacion_df.rename(columns={'longitude': 'longitud_destino','latitude': 'latitud_destino'}, inplace=True)

lineas_df.rename(columns={'id_header': 'id_header_A','name_header': 'name_origen_linea'}, inplace=True)
visualizacion_df =  visualizacion_df.merge(lineas_df[['id_header_A','name_origen_linea','latitude','longitude']], on='id_header_A',how='left')
visualizacion_df.rename(columns={'longitude': 'longitud_origen_linea','latitude': 'latitud_origen_linea'}, inplace=True)

lineas_df.rename(columns={'id_header_A': 'id_header_B','name_origen_linea': 'name_destino_linea'}, inplace=True)
visualizacion_df =  visualizacion_df.merge(lineas_df[['id_header_B','name_destino_linea','latitude','longitude']], on='id_header_B',how='left')
visualizacion_df.rename(columns={'longitude': 'longitud_destino_linea','latitude': 'latitud_destino_linea'}, inplace=True)
visualizacion_df.drop_duplicates(subset=['id_header_A','idunplug_station','id_header_B','idplug_station','num_viajes'],inplace=True)

In [6]:
visualizacion_df = visualizacion_df.reset_index()
visualizacion_df['identificador'] = visualizacion_df.index 
origen = visualizacion_df[[ 'id_line','idunplug_station','num_viajes','media','media_autobus','latitud_origen','longitud_origen','name_origen','identificador']]
destino = visualizacion_df[['id_line','idplug_station',  'num_viajes','media','media_autobus','latitud_destino','longitud_destino','name_destino','identificador']]
origen.columns = ['id_line','id_station','num_viajes','media','media_autobus','latitud', 'longitud','name_station_bici','identificador']
destino.columns = ['id_line','id_station','num_viajes','media','media_autobus','latitud', 'longitud','name_station_bici','identificador']
origen['punto'] = "origen"
destino['punto'] = "destino"

C:\Users\Arturo\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Arturo\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
aux = [origen,destino]

aux_df = pd.concat(aux)
aux_df.to_csv('visualizacion_copia.csv', sep=';', encoding='utf-8',index=False)

## lineas_autobus.csv

In [9]:
lineas = visualizacion_df[['id_line','id_header_A','name_origen_linea','latitud_origen_linea','longitud_origen_linea','id_header_B','name_destino_linea','latitud_destino_linea','longitud_destino_linea']]

In [10]:
lineas.drop_duplicates(subset=['id_header_A','id_header_B'],inplace=True)

C:\Users\Arturo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
o = lineas[['id_line','id_header_A','name_origen_linea','latitud_origen_linea','longitud_origen_linea']]
d = lineas[['id_line','id_header_B','name_destino_linea','latitud_destino_linea','longitud_destino_linea']]
o.columns = ['id_line','id_header','name_linea','latitud','longitud']
d.columns = ['id_line','id_header','name_linea','latitud','longitud']

aux = [o,d]
aux_df = pd.concat(aux)
aux_df.to_csv('lineas_autobus_copia.csv', sep=';', encoding='utf-8',index=False)